<a href="https://colab.research.google.com/github/santoshapatil/Edx-All-Courses/blob/main/edx_all_Courses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd


In [ ]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results

In [ ]:
course_name=[]
course_type=[]
course_photo_URL=[]
typ=[]
typ_name=[]
university_name=[]
university_logo=[]
course_link=[]
course_tit=[]
time_required=[]
course_level=[]
course_language=[]
course_subtitles=[]
course_skills=[]
category=[]
sub_category=[]
course_rating=[]

driver = webdriver.Chrome(options=options)
driver.implicitly_wait(5)
driver.get(url)

#calculating number of pages 
page_str = driver.find_element_by_xpath('(//button[@class="btn page-link"])[last()]').text
total_page = int(page_str)



In [ ]:
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(5)
driver.get(url)
page = driver.page_source
soup = BeautifulSoup(page, "html.parser")


In [ ]:
subj={'Architecture',
'Art %26 Culture',
'Biology %26 Life Sciences',
'Business %26 Management',
'Chemistry',
'Communication',
'Computer Science',
'Data Analysis %26 Statistics',
'Design',
'Economics %26 Finance',
'Education %26 Teacher Training',
'Electronics',
'Energy %26 Earth Sciences',
'Engineering',
'Environmental Studies',
'Ethics',
'Food %26 Nutrition',
'Health %26 Safety',
'History',
'Humanities',
'Language',
'Law',
'Literature',
'Math',
'Medicine',
'Music',
'Philanthropy',
'Philosophy %26 Ethics',
'Physics',
'Science',
'Social Sciences'}

In [ ]:
subj=['Architecture','Philanthropy']

In [ ]:
subj

In [ ]:
def get_course(u,n_p):
  print(u)
  total_page=n_p
  driver = webdriver.Chrome(options=options)
  driver.implicitly_wait(1)
  driver.get(u)
  page = driver.page_source
  soup = BeautifulSoup(page, "html.parser")
  for i in range(1,total_page+1):
  #driver = webdriver.Chrome(options=options)
  #driver.implicitly_wait(5)
  
    go=i
    c_xpath = '//div[@class="discovery-card Verified and Audit col col-xl-3 mb-4 static-card-spacing"]/a[@class="discovery-card-link"]'
    c_p_pg=len(soup.find_all('a',class_='discovery-card-link'))
    print(c_p_pg)
    for j in range(c_p_pg):
    
    #Attaching Course Link
      c_url=soup.find_all('a',class_='discovery-card-link')[j]
      clink="https://www.edx.org" + c_url.get('href')
      if clink in course_link:
        dup.append(clink)
        continue
      else:
        course_link.append(clink)
    
    #Attaching course Name/Title
      cnn=soup.find_all('h3',class_="name-heading")[j]
      if cnn is None:
        continue
      else:
        cnn=cnn.get_text()
        course_name.append(cnn)

    #attaching University /name
      univ = soup.find_all('div',class_='provider')[j]
      partner=univ.find_all('span')[-2].get_text()
      university_name.append(partner)
    
    #attaching Course photo
    
      try:

        c_pic=soup.find_all('div',class_='d-card-hero')[j]
        cp=c_pic.find('img').get('src')
        course_photo_URL.append(cp)
      except:
        course_photo_URL.append("not-mentioned")
    if i!=total_page:
      driver.find_element_by_xpath('//button[@class="btn next page-link"]').click()
    
      WebDriverWait(driver, 7).until(EC.visibility_of_element_located((By.XPATH, c_xpath)))
    #wait for another 30 seconds for the page to load completely
      time.sleep(6)
      #driver.get(url)
      go=go+1
      check = '//button[@aria-label="Page ' + str(go) + ', Current Page"]'
      page = driver.page_source
      soup = BeautifulSoup(page, "html.parser")
    else:
      break
  print("done")

In [ ]:
course_name=[]
course_type=[]
course_photo_URL=[]
typ=[]
typ_name=[]
university_name=[]
university_logo=[]
course_link=[]
course_tit=[]
time_required=[]
course_level=[]
course_language=[]
course_subtitles=[]
course_skills=[]
category=[]
sub_category=[]
course_rating=[]
dup=[]
for u in subj:
  subj_url="https://www.edx.org/search?subject="+u+"&tab=course"
  driver = webdriver.Chrome(options=options)
  driver.implicitly_wait(0.5)
  driver.get(subj_url)
  try:
    n_of_pgs = driver.find_element_by_xpath('(//button[@class="btn page-link"])')
    if n_of_pgs is None:
      get_course(subj_url,1)
    else:
      n_of_pgs = driver.find_element_by_xpath('(//button[@class="btn page-link"])[last()]').text
      n_of_pg = int(n_of_pgs)
      get_course(subj_url,n_of_pg)
  except NoSuchElementException :
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    
    c_p_pg=len(soup.find_all('a',class_='discovery-card-link'))
    print(c_p_pg)
    for j in range(c_p_pg):

    
    #Attaching course Name/Title
      cnn=soup.find_all('h3',class_="name-heading")[j]
      if cnn is None:
        continue
      else:
        cnn=cnn.get_text()
        course_name.append(cnn)
    
    #Attaching Course Link
      c_url=soup.find_all('a',class_='discovery-card-link')[j]
      clink="https://www.edx.org" + c_url.get('href')
      course_link.append(clink)

    #attaching University /name
      univ = soup.find_all('div',class_='provider')[j]
      partner=univ.find_all('span')[-2].get_text()
      university_name.append(partner)

    
    
    #attaching Course photo
    
      try:

        c_pic=soup.find_all('div',class_='d-card-hero')[j]
        cp=c_pic.find('img').get('src')
        course_photo_URL.append(cp)
      except:
        course_photo_URL.append("not-mentioned")


  

In [ ]:
len(dup)

3024

In [ ]:
len(course_link)

3082

In [ ]:
len(dup)

3024

In [ ]:
len(university_logo)

48

In [ ]:
edx_courses_catalog=pd.DataFrame(({'course_name':course_name,
                              'course_link':course_link,
                              'university_name':university_name,
                              'course_photo_URL':course_photo_URL
                              }))



In [ ]:
edx_courses_catalog

In [ ]:
#writing into a .csv file
edx_courses_catalog.to_csv('edx_courses_catalog.csv')

In [ ]:
edx_courses_catalog.university_logo.unique

In [ ]:
from google.colab import files
uploaded = files.upload()

import io
edx_courses_catalog = pd.read_csv(io.BytesIO(uploaded[r'edx_courses_catalog.csv']))
edx_courses_catalog.head()

In [ ]:
course_length=[]
course_level=[]
course_subtitle=[]
course_price=[]
course_effort=[]
course_language=[]
course_price=[]
course_enrollments=[]
course_partner=[]
university_link=[]
#for c,ind in zip(cs.course_link,cs.index):
for c,ind in zip(edx_courses_catalog.course_link,edx_courses_catalog.index):
  driver = webdriver.Chrome(options=options)
  driver.implicitly_wait(3)
  driver.get(c)
  page = driver.page_source
  soup = BeautifulSoup(page, "html.parser")
  print(ind)

   

#Attaching Course Enrollments
  try:
    enrol=soup.find('div',id="js-number-enrolled-label")
    enroll=enrol.find_all('span')[1].get_text()
    course_enrollments.append(enroll)
  except:
    course_enrollments.append("not-mentioned")
  try:
    check=soup.find_all('li',class_="list-group-item d-flex row px-0")
    leng,eff,lev,lng,sub,pric='','','','','',''
    for che in check:
      #Attaching Course Length
      t=che.get_text()
      if "Length"in t:
        leng=t.lstrip('Length:')
        course_length.append(leng)
      #Attaching Course Effort
      elif "Effort"in t:
        eff=t.lstrip('Effort:')
        course_effort.append(eff)
      #Attaching Course Level
      elif "Level"in t:
        lev=t.lstrip('Level:')
        course_level.append(lev)
      #Attaching Course Language
      elif "Language"in t:
        lng=t.lstrip('Language:')
        course_language.append(lng)
      #Attaching Course Subtitles
      elif "Transcript"in t:
        sub=t.lstrip('Video Transcript:')
        course_subtitle.append(sub)
      #Attaching Course Price
      elif "Price"in t:
        pric=t.lstrip('Price:FREE Add a Verified Certificate for')
        course_price.append(pric)
    
    if leng is '':
      course_length.append("not-mentioned")
    elif eff is '':
      course_effort.append("not-mentioned")
    elif lev is '':
      course_level.append("not-mentioned")
    elif lng is '':
      course_language.append("not-mentioned")
    elif sub is '':
      course_subtitle.append("not-mentioned")
    elif pric is '':
      course_price.append("not-mentioned")

  except:
    course_length.append("not-mentioned")
    course_effort.append("not-mentioned")
    course_level.append("not-mentioned")
    course_language.append("not-mentioned")
    course_subtitle.append("not-mentioned")
    course_price.append("not-mentioned")



  #Get University name and Partner Link
  try:
    univ=soup.find_all('div',class_='partner')
    universit=[]
    uni_link=[]
    for u in univ:
      ltp=u.find('a',class_='link').get_text()
      universit.append(ltp)
      univ_link="https://www.edx.org"+u.find('a').get('href')
      uni_link.append(univ_link)
    
    course_partner.append(universit)
    university_link.append(uni_link)
  except:
    course_partner.append("not-mentioned")
    university_link.append("not-mentioned")




In [ ]:
course_subtitle

['English', 'English']

In [ ]:
edx_courses_catalog_full=pd.DataFrame(({'course_name':course_name,
                              'course_link':course_link,
                              'university_name':university_name,
                              'course_photo_URL':course_photo_URL,
                              'course_level':course_level,
                              'course_effort':course_effort,
                              'course_length':course_length,
                              'course_language':course_language,
                              'course_subtitle':course_subtitle,
                              'course_price':course_price,
                              'course_partner':course_partner,
                              'university_link':university_link}))

In [ ]:
#Attaching Course Language
len(course_price)

1949

In [ ]:
course_partner

In [ ]:
course_length=[]
  fet=soup.find_all('div',class_='col d-flex')
  for f in fet:
    t=f.find('div',class_="col d-flex").get_text()
    if "Length"in t:
      leng=f.find("div",class_="col").get_text()
      course_length.append(leng)
    elif "Effort"in t:
      eff=f.find("div",class_="col").get_text()
      course_length.append(eff)
    elif "Level"in t:
      lev=f.find("div",class_="col").get_text()
      course_length.append(lev)
    elif "Language"in t:
      lng=f.find("div",class_="col").get_text()
      course_length.append(lng)
    elif "Transcript"in t:
      sub=f.find("div",class_="col").get_text()
      course_length.append(sub)